In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import argparse
import logging
import time
from os.path import join
import sys
from hist.intervals import ratio_uncertainty
sys.path.append('../')
import warnings
warnings.filterwarnings("ignore")

from coffea import processor, util

In [ ]:
from collections import defaultdict
import os
from os.path import join
import hist
import mplhep as hep
import seaborn as sns
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_systematic, plot_m4l_systematic, get_category_labels, get_color_list
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

# systematics to plot 
k_shifts = ["tauES", "efake", "mfake", "eleES", "eleSmear", "muES", "unclMET"]
e_shifts = ["l1prefire", "pileup", "btag"]

# filename info
year = "2018"
lumis = {"2018": 59.7}
source = "MC"

# grab a specific systematic
syst = "pileup"
plot = False
nom = util.load(f"../output/{source}_UL_2018_None_OS.coffea")
if syst in k_shifts:
    shift = util.load(f"../output/{source}_UL_2018_{syst}_OS.coffea")
else:
    shift = util.load(f"../output/{source}_UL_2018_None_OS.coffea")

cat_labels = get_category_labels()
colors = get_color_list()

outdir = f"../plots/systematics/{year}"

In [ ]:
var = "m4l"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:            
        sign = 0 # OS
        nom_s = sum(nom[var].values())
        shift_s = sum(shift[var].values())
        # group, cat, sign, mass_type, btags, syst, values
        nom_s = nom_s[::sum, 'ee'+cat, sign, :, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, sign, :, ::sum, 'nom', :]
        shift_up = shift_s[::sum, 'ee'+cat, sign, :, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, sign, :, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, sign, :, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, sign, :, ::sum, syst+"_down", :]
        plot_m4l_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, mass_label=r"$m_{ll\tau\tau}$ [GeV]", 
            logscale=True,
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "mtt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, :, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, :, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, :, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, :, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_down", :]
        plot_m4l_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, mass_label=r"$m_{\tau\tau}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        # group, cat, sign, btag, syst, vals
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down,
            syst, cat_label, r"$m_{ll}$ [GeV]", 
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        for leg in ['3', '4']:
            nom_s = sum(nom[var].values())
            nom_s = nom_s[::sum, 'ee'+cat, 0, leg, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, leg, ::sum, 'nom', :]
            shift_s = sum(shift[var].values())
            shift_up = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_up", :]
            shift_down = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_down", :]
            plot_systematic(
                nom_s, shift_up, shift_down, 
                syst, cat_label, f"Leg {leg} " + r"$p_T$ [GeV]",
                year=int(year), lumi = lumis[year],
                outfile = join(outdir, f"{source}_{cat}_{syst}_{var}{leg}.pdf") if plot else None
            )

In [ ]:
var = "met"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, r"$E_T^\mathrm{miss}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf")
        )